In [1]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
import io

Init Plugin
Init Graph Optimizer
Init Kernel


In [17]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")

vocab_size = pd.get_vocabulary_size()
window_size = 3
num_ns = 5
embedding_dim = 64
embedding_layer_name = "n2v_embedding"

In [9]:
db = DatasetBuilder("dataset_objects/full_corpus")

In [10]:
# takes around 66 mins for window_size = 3, num_ns = 5
dataset = db.build_word_to_vec_dataset(
        vocab_size, window_size, num_ns, skip_amount=1)

100%|██████████| 2200/2200 [1:05:32<00:00,  1.79s/it]


In [18]:
n2v = build_note_to_vec_model(
        vocab_size, embedding_dim, num_ns, embedding_layer_name)

n2v.summary()

In [19]:
# takes around 11 mins for embedding_dim = 32
n2v.fit(dataset, epochs=20)

Epoch 1/20
  13/5136 [..............................] - ETA: 45s - loss: 1.7857 - accuracy: 0.4458

2021-10-12 17:16:58.587583: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5136/5136 [==============================] - 34s 7ms/step - loss: 0.2855 - accuracy: 0.9095
Epoch 2/20
5136/5136 [==============================] - 34s 7ms/step - loss: 0.2357 - accuracy: 0.9183
Epoch 3/20
5136/5136 [==============================] - 34s 7ms/step - loss: 0.2354 - accuracy: 0.9184
Epoch 4/20
5136/5136 [==============================] - 35s 7ms/step - loss: 0.2353 - accuracy: 0.9184
Epoch 5/20
5136/5136 [==============================] - 39s 8ms/step - loss: 0.2351 - accuracy: 0.9184
Epoch 6/20
5136/5136 [==============================] - 37s 7ms/step - loss: 0.2349 - accuracy: 0.9185
Epoch 7/20
5136/5136 [==============================] - 35s 7ms/step - loss: 0.2345 - accuracy: 0.9186
Epoch 8/20
5136/5136 [==============================] - 34s 7ms/step - loss: 0.2343 - accuracy: 0.9187
Epoch 9/20
5136/5136 [==============================] - 34s 7ms/step - loss: 0.2343 - accuracy: 0.9188
Epoch 10/20
5136/5136 [==============================] - 34s 7ms/step - loss: 0.2342

In [20]:
vocab = pd.get_vocabulary()
weights = n2v.get_layer(embedding_layer_name).get_weights()[0]

In [21]:
vectors_file = io.open('vectors_64.tsv', 'w', encoding='utf-8')
metadata_file = io.open('metadata_64.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    vec = weights[index]
    vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")
    metadata_file.write(word + "\n")

vectors_file.close()
metadata_file.close()